In [1]:
from selenium import webdriver  # allows you to launch a browser
from selenium.webdriver.common.by import By  # allows you to search for things using specific parameters
from selenium.webdriver.support.ui import WebDriverWait  # allows you to wait for a page to load
from selenium.webdriver.support import expected_conditions as EC   
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import requests
import time

from urllib.request import Request, urlopen
import os
import re
import urllib.request, urllib.error, urllib.parse
import mysql.connector
import warnings
import json
import codecs

In [14]:
# This function starts by opening the Chrome Web Broswer and use Selenium package to have it

# These 3 lines set up Chrome option as Incognito
# Open an empty Chrome browser

option = webdriver.ChromeOptions()
option.add_argument(" — incognito")         #browse in incognito mode
driver = webdriver.Chrome('C:\Windows\chromedriver')                   # open chrome

# This function go to the CARVANA search page and enter Zipcode
# As well as close potential pop up windows.

def enter_zipcode():                
    driver.get("https://www.carvana.com/cars/bmw?geolocation")    # open the website in Chrome Browser
    
    time.sleep(10)                              # wait for the website to load
    
    search_bar = driver.find_element_by_name('ZIP CODE')    # This is the search bar for ZIPCODE
    
    search_bar.click()                          # click the search bar
    
    for i in range(6):                          # delete the prefilled ZIPCODE, if any
        search_bar.send_keys(Keys.BACKSPACE)
        
    search_bar.send_keys("94103")               # send the San Francisco Zip code to the search bar
    
    time.sleep(5)
    
    search_bar.send_keys(Keys.RETURN)           # Press Enter
    
    time.sleep(10)                              # wait for page to load
    
    
    try:
        # In case the zip search window does not automatically close
        # find the close button and close it
        button2 = driver.find_element_by_xpath('//div[@class="Modal__HeaderContainer-sc-150123d-1 jEKQOU"]/*[name()="svg"][@data-test="SvgIcon"]')
        button2.click()
        print("We have finished searching for cars near San Francisco, 94103")
            
    except:
        print("We have finished searching for cars near San Francisco, 94103")
        
        
enter_zipcode()

We have finished searching for cars near San Francisco, 94103


In [15]:
# In case Carvana sends up email sign-up advertisement pop-up. Close it

def check_more_pop_ups():
    try:
        button = driver.find_element_by_class_name('close')    # In case Carvana send us a pop up
        button.click()
        
        print("We have closed all pop up from Carvana. Good to go")
        
    except:
        print('There does not seem to have any pop up left. Good to go')
        
check_more_pop_ups()

There does not seem to have any pop up left. Good to go


In [16]:
def create_folder():
    directory = "BMW_search_pages"                                         # This is the name of the file
    parent_dir = "C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1"                # This is the parent directory
    
    global path                                               
    path = os.path.join(parent_dir, directory)
    if not os.path.exists(path):
        os.mkdir(path)                                              
    
create_folder()

In [17]:
# This function asks Chrome browser to save page source codes for each page in files. And then click next page button.

def go_to_next_page_and_save_every_page():
    for i in range(1000):
        # write down the content on the current browser page
        j = i+1
        file = open(os.path.join(path, 'page{}'.format(j)), 'w', encoding = 'utf-8')
        file.write(driver.page_source)
        file.close()
        
        # Look for 'next' button
        action = 'start'
        next_page = driver.find_element_by_xpath("//button[@type='button' and @data-cv-test='Cv.Search.Pagination.NextPageButton']")
        try:
            # If 'next' button is disabled, means we are on the last page. Stop clicking
            disabled_button = driver.find_element_by_xpath("//button[@type='button' and @data-cv-test='Cv.Search.Pagination.NextPageButton' and @disabled]")
            action = 'stop'
            print("we have went through from the first page to the last page.")
        except:
            # If 'next' button is not disabled, click it and go to next page
            next_page.click()
            
        time.sleep(2)
            
        if action == 'stop':
            break
        
            

go_to_next_page_and_save_every_page()

we have went through from the first page to the last page.


In [22]:
def extract_car_url():
    # check if file already exists. If so open and write. If not, create one
    
    if os.path.isfile("C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_url.txt"):
        url_file = open("C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_url.txt", "w")
    else:
        url_file = open("C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_url.txt", "x")
    
    #Write all car URL into the file
    for file in os.listdir(path):
        with open(os.path.join(path, file), 'r', encoding = 'utf-8') as f:
            contents = f.read()
            soup = BeautifulSoup(contents, 'lxml')
            
            links = soup.select("article[class=result-tile] > a")
            for l in links:
                url_file.write('https://www.carvana.com' + l.get('href') + '\n')
            
    url_file.close()
                
extract_car_url()

In [45]:
# create a folder to save all the page contents of car urls

def create_second_folder():
    directory = "car_listing_details"                                         # This is the name of the file
    parent_dir = "C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1"                # This is the parent directory
    
    global path2                                               
    path2 = os.path.join(parent_dir, directory)  
    if not os.path.exists(path2):
        os.mkdir(path2)                                           
    
create_second_folder()

In [12]:
#path2 = "C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_listing_details"

In [24]:
# This function download page content from each BMW car listing
# saves the content in files named by the VehicleID
# All the files can be used for BeautifulSoup in the future

# Need to pause 5 second between each request. Avoid being blocked by security
# This function will run for almost.


def get_content_from_car_url():
    f = open("C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_url.txt", 'r')
    
    option = webdriver.ChromeOptions()
    option.add_argument(" — incognito")         #browse in incognito mode
    driver = webdriver.Chrome('C:\Windows\chromedriver')                   # open chrome
    
    for url in f:
        try:
            driver.get(url)
            
            time.sleep(5)
            
            # Scroll down the page to load more contents
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            time.sleep(3)
            
            content = driver.page_source
            
            if os.path.isfile("C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_listing_details\\{}.txt".format(url[-8:-1])):
                file = open("C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_listing_details\\{}.txt".format(url[-8:-1]), 'w', encoding = 'utf-8')
            else:
                file = open("C:\\Users\\jayem\\Documents\\UC Davis MSBA\\WINTER QUARTER 2020\\BAX 422 Section 1\\car_listing_details\\{}.txt".format(url[-8:-1]), 'x', encoding = 'utf-8')
            file.write(content)
            file.close()
        
        except:
            print(url + 'is not longer valid. Car is Sold')
            
    driver.close()  

In [25]:
get_content_from_car_url()

https://www.carvana.com/vehicle/1657367
is not longer valid. Car is Sold


In [19]:
# Create a data base in MySQL

def create_database():
    conn = mysql.connector.connect(user='root', password='horseratio', host = 'localhost')
    cursor = conn.cursor()
    cursor.execute("CREATE database if not exists CARVANA")
    cursor.execute("USE CARVANA")
    
    cursor.execute("CREATE table if not exists bmwsearch (\
    id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, \
    VIN VARCHAR(20), \
    StockNo INT(20), \
    model VARCHAR(50), \
    series VARCHAR(500), \
    price INT(20), \
    tax_title_reg INT(20), \
    mileage INT(20), \
    cylinder_count INT(20), \
    engine_desc VARCHAR(50), \
    horsepower INT(20),  \
    torque INT(20),  \
    fuel_description VARCHAR(50), \
    drive_trains VARCHAR(50), \
    seats INT(20),  \
    mpg_city INT(20), \
    mpg_hwy INT(20), \
    ext_color VARCHAR(50), \
    int_color VARCHAR(50), \
    transmission1 VARCHAR(50), \
    transmission2 VARCHAR(1000), \
    doors INT(20))")
    
    #numkeys INT(20), \
    #imperfections INT(20), \
    #int_appearance VARCHAR(50), \
    #photo_count INT(20), \
    #saves INT(20), \
    #views INT(20), \
    #sound_system BOOLEAN, \
    #driver_assitance BOOLEAN, \
    #keyless_go BOOLEAN, \
    #cruise_control BOOLEAN, \
    #heated_seats BOOLEAN, \
    #xenon_lamps BOOLEAN, \
    #pano_roof BOOLEAN)")

    conn.commit()
    conn.close()

In [5]:
create_database()

In [42]:
prepared_stmt = "INSERT INTO BMWSEARCH" + " (VIN,StockNo,model,series,price,tax_title_reg,mileage,cylinder_count,engine_desc,horsepower,torque,fuel_description,drive_trains,seats,mpg_city,mpg_hwy,ext_color,int_color,transmission1,transmission2,doors) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
#prepared_stmt = "INSERT INTO BMWSEARCH" + " (VIN,StockNo,model,series,price,tax_title_reg,mileage,cylinder_count,engine_desc,horsepower,torque,fuel_description,drive_trains,seats,mpg_city,mpg_hwy,ext_color,int_color,transmission1,transmission2,doors) values ({},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{});"

In [44]:
VINs = []
StockNos = []
models = []
series = []
prices = []
ttrs = []
mileages = []
cyl_counts = []
engine_descs = []
horsepowers = []
torques = []
fuel_descs = []
drive_trains = []
seats = []
mpg_city = []
mpg_hwy = []
ext_colors = []
int_colors = []
transmission1 = []
transmission2 = []
doors = []
great_deals = []
saved = []
view = []
pictures = []
imperfections = []
i = 1
conn = mysql.connector.connect(user='root', password='horseratio', database = 'carvana', host = 'localhost')
cursor=conn.cursor()
## GET LISTS OF ALL VARIABLES TO SCRAPE
for filename in os.listdir(path2):
    with open(os.path.join(path2,filename), encoding="utf8") as openfile:          
        content = openfile.read()
        soup = BeautifulSoup(content, 'html.parser')
        try:
            VIN = re.search("VIN: .{17}", content)
            if VIN == None: VIN = ''
            else: 
                VIN = VIN.group(0)
                VIN = VIN[5:]
            VINs.append(VIN)
            StockNo = re.search("Stock #: [0-9]{10}", content)
            if StockNo == None: StockNo = 0
            else:
                StockNo = StockNo.group(0)
                StockNo = StockNo[9:]
            StockNos.append(StockNo)
            model = re.search("MODEL <\/span>(.+?)<\/p>{1}", content)
            if model == None: model = ''
            else:
                model = model.group(0)
                model = model[13:-4]
            models.append(model)
            seriesname = re.search("(ColumnOneNonMobilestyles__VehicleInfoText-sc-1ab7xgo-1 ZXcSi\">(.*?)){2}(.*?)(<\/p>)", content)
            if seriesname == None: seriesname = ''
            else: 
                seriesname = seriesname.group(0)
                seriesname = seriesname[(seriesname.rfind(">", 122, len(seriesname)-1)+1): -4]
            series.append(seriesname)
            price = re.search("Vehicle Price(.+?)(\$[0-9]+,[0-9]{3})", content)
            if price == None:
                price = re.search("menu-price-text\">(\$[0-9]+,[0-9]{3})",content)
                if price == None: price = 0
                else: 
                    price = price.group(0)
                    price = price[price.rfind("$"):]
                    price = price.replace("$","")
                    price = price.replace(",","")
            else: 
                price = price.group(0)
                price = price[price.rfind("$"):]
                price = price.replace("$","")
                price = price.replace(",","")
            prices.append(price)
            taxtitlereg = re.search("Tax, Title, &amp; Reg.(.+?)(\$[0-9,]+)",content)
            if taxtitlereg == None: taxtitlereg = 0
            else: 
                taxtitlereg = taxtitlereg.group(0)
                taxtitlereg = taxtitlereg[taxtitlereg.rfind("$"):]
                taxtitlereg = taxtitlereg.replace("$","")
                taxtitlereg = taxtitlereg.replace(",","")
            ttrs.append(taxtitlereg)
            mileage = re.search("([0-9,]+) miles", content)
            if mileage == None: mileage = 0
            else:
                mileage = mileage.group(0)
                mileage = mileage[:-6]
                mileage = mileage.replace(",","")
            mileages.append(mileage)
            cyl_count = re.search("CylinderCount\":[0-9]+", content)
            if cyl_count == None: cyl_count = 0
            else: 
                cyl_count = cyl_count.group(0)
                cyl_count = cyl_count[-1:]
            cyl_counts.append(cyl_count)
            enginedesc = re.search("engineDescription\":\"(.+?)\",",content)
            if enginedesc == None: enginedesc = ''
            else:
                enginedesc = re.search("engineDescription\":\"(.+?)\",",content).group(0)
                enginedesc = enginedesc.replace("engineDescription\":\"","")
                enginedesc = enginedesc.replace("\",","")
            engine_descs.append(enginedesc)
            horsepower = re.search("engineHorsepower\":[0-9]+",content)
            if horsepower == None: horsepower = 0
            else: 
                horsepower = horsepower.group(0)
                horsepower = horsepower[horsepower.rfind(":")+1:]
            horsepowers.append(horsepower)
            torque = re.search("engineTorque\":[0-9]+",content)
            if torque == None: torque = 0
            else: 
                torque = torque.group(0)
                torque = torque[torque.rfind(":")+1:]
            torques.append(torque)
            fuel_desc = re.search("fuelDescription\":\"[A-Za-z]+",content)
            if fuel_desc == None: fuel_desc = ''
            else:
                fuel_desc = fuel_desc.group(0)
                fuel_desc = fuel_desc[fuel_desc.rfind(":")+2:]
            fuel_descs.append(fuel_desc)
            drive_train = re.search("driveTrainDescription\":\"[A-Za-z]+",content)
            if drive_train == None: drive_train = ''
            else:
                drive_train = drive_train.group(0)
                drive_train = drive_train[drive_train.rfind(":")+2:]
            drive_trains.append(drive_train)
            seat = re.search("seating\":\"[0-9]+",content)
            if seat == None: seat = 0
            else:
                seat = seat.group(0)
                seat = seat[-1:]
            seats.append(seat)
            mpgc = re.search("mpgCity\":[0-9]+",content)
            if mpgc == None: mpgc = 0
            else:
                mpgc = mpgc.group(0)
                mpgc = mpgc[mpgc.rfind(":")+1:]
            mpg_city.append(mpgc)
            mpgh = re.search("mpgCity\":[0-9]+",content)
            if mpgh == None: mpgh = 0
            else:
                mpgh = mpgh.group(0)
                mpgh = mpgh[mpgh.rfind(":")+1:]
            mpg_hwy.append(mpgh)
            ext_color = re.search("exteriorColor\":\"[A-Za-z]+",content)
            if ext_color == None: ext_color = ''
            else: 
                ext_color = ext_color.group(0)
                ext_color = ext_color[ext_color.rfind(":")+2:]
            ext_colors.append(ext_color)
            int_color = re.search("interiorColor\":\"[A-Za-z ]+",content)
            if int_color == None: int_color = ''
            else: 
                int_color = int_color.group(0)
                int_color = int_color[int_color.rfind(":")+2:]
            int_colors.append(int_color)
            transmission = re.search("transmission\":\"(.+?)\"",content)
            if transmission == None: 
                trans1 = ''
                trans2 = ''
            else:
                transmission = transmission.group(0)
                transmission = transmission.replace("transmission\":\"","")
                trans1 = re.search(".+,",transmission)
                if trans1 == None: 
                    trans1 = ''
                    trans2 = ''
                else:
                    trans1 = trans1.group(0)
                    trans1 = trans1[:-1]
                    if re.search("[0-9]",transmission) == None: trans2 = ''
                    else:
                        trans2 = transmission[transmission.rfind(re.search("[0-9]",transmission).group(0)):-1]
            transmission1.append(trans1)
            transmission2.append(trans2)
            door = re.search("doors\":[0-9]",content)
            if door == None: door = 0
            else:
                door = door.group(0)
                door = door[-1:]
            doors.append(door)
            great_deal = re.search("Great Deal", content)
            if great_deal == None: great_deal = False
            else: great_deal = True
            great_deals.append(great_deal)
            saved_count = soup.findAll('span', {'class': 'count'})
            if saved_count == []:
                saved_count = 0
            else:
                saved_count = soup.findAll('span', {'class': 'count'})[0].text
            saved.append(saved_count)                
            view_count = soup.findAll('span', {'class': 'count'})
            if view_count == []:
                view_count = 0
            else:
                view_count = soup.findAll('span', {'class': 'count'})[1].text
            view.append(saved_count)                
            pic = re.search("Slide 1 of [0-9]+",content)
            if type(pic) == re.Match:
                pic = re.search("Slide 1 of [0-9]+",content).group(0)[-2:]
                if pic == " ":
                    pic = 0
                else:
                    pic = pic             
            else:
                pic = 0
            pictures.append(pic)
            #Each car has at least 1 imperfection
            imp = soup.find('a', {'class': 'ImperfectionsText__LinkText-sc-9swxmp-2 frWjah'})
            if imp == None:
                imp = 0     
            else: 
                imp = imp.text[5]
                if imp == " ":
                    imp = 1
                else:
                    imp = imp
            imperfections.append(imp)
        except Exception:
            #seriesname = ''
            #series.append(seriesname)
            print(i)
            pass
        openfile.close()
        i = i+1
        cursor.execute(prepared_stmt, (VIN,StockNo,model,seriesname,price,taxtitlereg,mileage,cyl_count,enginedesc,horsepower,torque,fuel_desc,drive_train,seat,mpgc,mpgh,ext_color,int_color,trans1,trans2,door,great_deal,saved_count,view_count,pic,imp))
conn.commit()
cursor.close()
conn.close()

In [46]:
# Variable list check

varlist = [VINs,StockNos,models,series,prices,ttrs,mileages,cyl_counts,engine_descs,horsepowers,torques,fuel_descs,drive_trains,
          seats,mpg_city,mpg_hwy,ext_colors,int_colors,transmission1,transmission2,doors]

for l in varlist: print(len(l))

661
661
661
661
661
661
661
661
661
661
661
661
661
661
661
661
661
661
661
661
661


In [20]:
# test block to ensure individual variables are pulling from files
i = 1
test = []
for filename in os.listdir(path2):
    with open(os.path.join(path2,filename), encoding="utf8") as openfile:            
        content = openfile.read()
        try: 
            VIN = re.search("VIN: .{17}", content)
            if VIN == None: VIN = ''
            else: 
                VIN = VIN.group(0)
                VIN = VIN[5:]
            print(VIN)
            print(i)
            i = i + 1
            test.append(VIN)
    
        except Exception:
            #seriesname = 'hi'
            i = i+1
            pass
        openfile.close()

WBXHT3C3XGP889847
1
WBY1Z4C53EVX62966
2
WBA3B1C51DF462524
3
WBA3A9G56DNP36978
4
5UXZW0C56CL672773
5
WBA8E1G58HNU15400
6
WBY1Z4C51GV508000
7
WBA3B1G5XFNT04217
8
5UXXW7C51H0U25542
9
WBA3N7C55EK221861
10
WBA3X5C51ED556621
11
WBY1Z2C55EVX51049
12
WBY1Z2C53EVX51633
13
5UXKR0C31H0X82887
14
WBY1Z2C50EV284526
15
5UXWX9C51G0D83668
16
WBS3R9C53FK334866
17
WBA3B1G57FNT03896
18
5UXWX9C50D0D00212
19
WBAPL5G51BNN23017
20
WBY1Z4C52FV279289
21
5UXWX7C57H0S19960
22
WBAKE5C53DE432375
23
WBA3A5C55DF601385
24
WBA3A5C5XFF609775
25
WBAXG5C59CDX04352
26
WBA8B9G52HNU51431
27
WBA5B1C54ED482448
28
WBAVL1C59EVY20949
29
WBXHU7C38J5H41755
30
WBA3C1C53EK106193
31
5UXXW3C57F0M89221
32
WBALZ3C57CDL72795
33
WBAJR7C04LWW65058
34
WBA1F9C57FVW98632
35
WBSKG9C56BE797241
36
WBA5B1C5XED481966
37
WBA3R1C53EK190109
38
WBA4A9C50FGL86894
39
WBAXH5C51DD106514
40
WBA3B9G59ENR90697
41
WBA3A5G53FNS89649
42
5UXWX9C52H0T08637
43
WBA3B1C54EP679979
44
WBA3A5C56CF345031
45
5UXKU2C54G0N80717
46
WBAKF5C56CE656591
47
WBXHU7C33H5H33413
48
W